# Can you quantify the culture of a city? 
## Studying the many moods of San Francisco through movies, using Python, Pandas, and Plotly:


*“Cities, like dreams, are made of desires and fears, even if the thread of their discourse is secret, their rules are absurd, their perspectives deceitful, and everything conceals something else.”*
― Italo Calvino, Invisible Cities 

In this notebook I hope to reveal some of the threads of San Francisco's outline, as used for dramatic effect in films. The city government of San Francisco keeps records, going back nearly to the dawn of film, of all the locations of the city that have been featured on celluloid. The data is messy, so I perform data cleaning and pre-processing before mapping the data and examining its structure. 

Can we quantify the "mood" of a neighborhood based on the kinds of movies that have used it for locations? We will find out!
Special thanks to the city of San Francisco for making these records readily available!

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import scipy as sc
import requests
import io
import gzip

!pip install geopandas greppo sodapy bs4 thefuzz[speedup] cinemagoer

from imdb import Cinemagoer

from thefuzz import fuzz, process #https://github.com/seatgeek/thefuzz, updated version of fuzzy wuzzy
from bs4 import BeautifulSoup
from re import search
import geopandas
import greppo
import sodapy
from sodapy import Socrata

import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

import tqdm
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.3 MB/s 
     |████████████████████████████████| 1.7 MB 54.9 MB/s 
     |████████████████████████████████| 301 kB 64.7 MB/s 
     |████████████████████████████████| 16.7 MB 419 kB/s 
     |████████████████████████████████| 6.3 MB 37.1 MB/s 
     |████████████████████████████████| 97 kB 4.8 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 19.3 MB 1.3 MB/s 
     |████████████████████████████████| 3.1 MB 65.9 MB/s 
     |████████████████████████████████| 61 kB 290 kB/s 
     |████████████████████████████████| 1.0 MB 41.8 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 80 kB 10.4 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Created wheel for meta: filename=meta-1.0.

# Import Data

### Create connection to SF main database
We use sodapy to access the API

In [ ]:
#used to use sodapy to access the SODA API
sf_database_api = 'data.sfgov.org'
client = Socrata(sf_database_api, None) #use without token

### Import database of all street addresses in San francisco
Available at: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Addresses-Enterprise-Addressing-System/3mea-di5p

In [ ]:
#if you're paging, make sure you provide an $order clause or at a minimum $order=:id.
#That will guarantee that the order of your results will be stable as you page through the dataset. 
sf_address_API_key = "3mea-di5p"

all_address_size = client.get(sf_address_API_key, select = 'COUNT(*)') #return the count of all rows in the dataset, which = dataset size 
all_address_size= int(all_address_size[0]['COUNT'])
limit_size = 5000
street_stride = np.arange(0, all_address_size, limit_size) #this will be used to page through the SF address database with a given pagination chunk

all_address_metadata = client.get_metadata(sf_address_API_key)
all_sf_address_cols = [x['name'] for x in all_address_metadata['columns']]

print(' Total Number of recorded street addresses in SF: ', all_address_size)

 Total Number of recorded street addresses in SF:  223182


In [ ]:
df_all_sf_address_list = list()
for stride_num, stride_start in enumerate(street_stride):

    sf_address_all_request = client.get(sf_address_API_key, limit = limit_size, offset = stride_start, order = 'Address') #without specifying, this will only return 1000 results, need to augment explicitly 
    df_all_sf_address_list.append(pd.DataFrame(sf_address_all_request).drop(['supervisor', 'supname'], axis = 1))

In [ ]:
df_all_sf_address = pd.concat(df_all_sf_address_list).reset_index() #concat list of DFs 
assert df_all_sf_address.address.count() == all_address_size, f"concat DF size equal to count of all database records expected, got: {df_all_sf_address.address.count()}"


In [ ]:
print('Number of addresses listed in this table is ', df_all_sf_address.address.count())
df_all_sf_address.loc[:, 'street_name']  = df_all_sf_address.loc[:, 'street_name'].str.lower()
df_all_sf_address.loc[:, 'address']  = df_all_sf_address.loc[:, 'address'].str.lower()
df_all_sf_address.loc[:, 'street_type']  = df_all_sf_address.loc[:, 'street_type'].str.lower()
df_all_sf_address.head(3).append(df_all_sf_address.tail(3))


Number of addresses listed in this table is  223182


,index,eas_baseid,address,address_number,street_name,street_type,zip_code,cnn,longitude,latitude,point,supdist,supdistpad,numbertext,nhood,address_number_suffix
0,0,383793,0 alemany blvd,0,alemany,blvd,94110,2156000,-122.41038038,37.73526758,"{'type': 'Point', 'coordinates': [-122.4103803...",SUPERVISORIAL DISTRICT 10,10,TEN,Portola,NaN
1,1,310771,0 balboa st,0,balboa,st,94121,2672000,-122.48717207,37.7756991,"{'type': 'Point', 'coordinates': [-122.4871720...",SUPERVISORIAL DISTRICT 2,02,TWO,Golden Gate Park,NaN
2,2,454709,0 bayview park rd,0,bayview park,rd,94124,2836000,-122.39510433,37.71751859,"{'type': 'Point', 'coordinates': [-122.3951043...",SUPERVISORIAL DISTRICT 9,09,NINE,Visitacion Valley,NaN
223179,3179,342858,9 woodland ave,9,woodland,ave,94117,13710000,-122.45367508,37.76395976,"{'type': 'Point', 'coordinates': [-122.4536750...",SUPERVISORIAL DISTRICT 5,05,FIVE,Inner Sunset,NaN
223180,3180,379766,9 wright st,9,wright,st,94110,13736000,-122.40550365,37.74721756,"{'type': 'Point', 'coordinates': [-122.4055036...",SUPERVISORIAL DISTRICT 10,10,TEN,Bernal Heights,NaN
223181,3181,371898,9 young ct,9,young,ct,94124,13786000,-122.38616702,37.73646718,"{'type': 'Point', 'coordinates': [-122.3861670...",SUPERVISORIAL DISTRICT 10,10,TEN,Treasure Island,NaN


In [ ]:
df_all_sf_address['street_name'].loc[df_all_sf_address['street_name'].isin(['the embarcadero'])] = 'embarcadero'

#### Import movie location data

This project will use data provided by Data SF, specifically a database of Film Locations in San francisco starting from 1924, provided by the San Francisco Film Commission.

For more information see:https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am


In [ ]:
#https://dev.socrata.com/foundry/data.sfgov.org/yitu-d5am for more information on querying
#Dataset Identifier: yitu-d5am

In [ ]:
movie_location_results = client.get("yitu-d5am", limit=5000) 

In [ ]:
df_movie_location = pd.DataFrame(movie_location_results)
df_movie_location.head(3).append(df_movie_location.tail(3))

,title,release_year,locations,production_company,director,writer,actor_1,actor_2,actor_3,distributor,fun_facts
0,180,2011,Epic Roasthouse (399 Embarcadero),SPI Cinemas,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand,NaN,NaN
1,180,2011,Mason & California Streets (Nob Hill),SPI Cinemas,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand,NaN,NaN
2,180,2011,Justin Herman Plaza,SPI Cinemas,Jayendra,"Umarji Anuradha, Jayendra, Aarthi Sriram, & Suba",Siddarth,Nithya Menon,Priya Anand,NaN,NaN
1973,Women is Losers,2020,Balmy Alley,Look at the Moon Pictures,Lissette Feliciano,Lissette Feliciano,Lorenza Izzo,Simu Liu,Liza Weil,NaN,NaN
1974,"Yours, Mine and Ours",1968,NaN,Desilu Productions,Melville Shavelson,"Bob Carroll, Jr.",Lucille Ball,Henry Fonda,Van Johnson,United Artists,NaN
1975,Zodiac,2007,SF Chronicle Building (901 Mission St),Paramount Pictures,David Fincher,James Vanderbilt,Jake Gyllenhaal,Mark Ruffalo,NaN,Paramount Pictures,NaN


In [ ]:
# TODO: word cloud by location?
key_IDs = ['title', 'release_year', 'locations'] #list of important fields i want to use
df_movie_basic_info = df_movie_location.loc[:,key_IDs]
df_movie_basic_info.loc[:, 'locations'] = df_movie_basic_info.loc[:, 'locations'].str.lower()
df_movie_basic_info.head(3)

,title,release_year,locations
0,180,2011,epic roasthouse (399 embarcadero)
1,180,2011,mason & california streets (nob hill)
2,180,2011,justin herman plaza


For future reference, I take any NaN (not a number) values in the dataframe and rename them to the string "Empty", for future reference.

In [ ]:
#need to clean NAN in this list, rename every NaN as empty as flag to skip
df_movie_basic_info[df_movie_basic_info.isnull()] = 'Empty'
df_movie_basic_info

,title,release_year,locations
0,180,2011,epic roasthouse (399 embarcadero)
1,180,2011,mason & california streets (nob hill)
2,180,2011,justin herman plaza
3,180,2011,200 block market street
4,180,2011,city hall
...,...,...,...
1971,Women is Losers,2020,3202 folsom st
1972,Women is Losers,2020,110 pierce st
1973,Women is Losers,2020,balmy alley
1974,"Yours, Mine and Ours",1968,Empty


### Import real landmark list and addresses from wikipedia

Some of the places listed in the 'locations' tab are landmarks, not street addresses (eg. City Hall). Therefore, I scrape a wikipedia list of landmarks in SF to hopefully transform landmark-only locations into street addresses.



In [ ]:
# use requests library to pull from wikipedia
sf_landmarks_wiki_url = 'https://en.wikipedia.org/wiki/List_of_San_Francisco_Designated_Landmarks'
wiki_response = requests.get(sf_landmarks_wiki_url)
print(wiki_response)

<Response [200]>


In [ ]:
#use beautiful soup to parse html
soup = BeautifulSoup(wiki_response.text, 'html.parser')
table = soup.find('table', {'class': 'wikitable'})

df_table_string = pd.read_html(str(table))
df_landmark_wiki = pd.DataFrame(df_table_string[0])
df_landmark_wiki.drop(['Image'], axis = 1, inplace = True)
df_landmark_wiki.rename(columns = {'Name': 'Landmark Name'}, inplace = True)
#string formatting step
df_landmark_wiki.loc[:, 'Landmark Name'] =df_landmark_wiki.loc[:, 'Landmark Name'].str.lower()
df_landmark_wiki.loc[:, 'Address'] =df_landmark_wiki.loc[:, 'Address'].str.lower()

df_landmark_wiki.head().append(df_landmark_wiki.tail())

,#,Landmark Name,Address,Date,Description[a]
0,1,mission dolores,320 dolores st.,4/11/68,".mw-parser-output .geo-default,.mw-parser-outp..."
1,2,old saint mary's cathedral,660 california st.,4/11/68,37°47′34″N 122°24′21″W﻿ / ﻿37.79265°N 122.40575°W
2,3,bank of california building,400 california st.,9/3/68,37°47′36″N 122°24′06″W﻿ / ﻿37.7932°N 122.4016°...
3,4,saint patrick's church,756 mission st.,9/3/68,37°47′08″N 122°24′13″W﻿ / ﻿37.785588°N 122.403...
4,5,saint francis of assisi church,610 vallejo st.,9/3/68,37°47′56″N 122°24′28″W﻿ / ﻿37.7988°N 122.4077°W
277,279,arthur h. coleman medical center,6301 3rd street,2018-09-03,[27]
278,280,new pullman hotel,228-248 townsend strret,2018-12-02,[28]
279,281,"piledrivers, bridge, and structural ironworkers",281-247 bryant street,2018-12-02,[29]
280,282,hotel utah,500-504 fourth street,2018-12-02,[30]
281,283,"dunham, carrigan & hayden building",2 henry adams,2019-01-25,[31]


#### Import table of all business names in san francisco

https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis


In [ ]:

sf_business_API_key = "g8m3-pdis"

all_business_size = client.get(sf_business_API_key, select = 'COUNT(*)') #return the count of all rows in the dataset, which = dataset size 
all_business_size= int(all_business_size[0]['COUNT'])
limit_size = 5000
street_stride = np.arange(0, all_business_size, limit_size) #this will be used to page through the SF business address database with a given pagination chunk

df_all_sf_business_list = list()
for stride_num, stride_start in enumerate(street_stride):
    df_all_sf_business_stride = client.get(sf_business_API_key, limit = limit_size, offset = stride_start, order = "ttxid") #without specifying, this will only return 1000 results, need to augment explicitly 
    df_all_sf_business_list.append(pd.DataFrame(df_all_sf_business_stride))

In [ ]:
print(all_business_size, len(df_all_sf_business_list), len(df_all_sf_business_list[1]))
df_all_sf_business_list = pd.concat(df_all_sf_business_list).reset_index() #concat list of DFs 

291211 59 5000


In [ ]:
df_all_sf_business_list.head(2)

,index,ttxid,certificate_number,ownership_name,dba_name,full_business_address,city,state,business_zip,dba_start_date,...,transient_occupancy_tax,location,uniqueid,supervisor_district,neighborhoods_analysis_boundaries,location_end_date,dba_end_date,lic,lic_code_description,business_corridor
0,0,0000024-02-999,0000024,Eastman Kodak Co,Eastman Kodak Co,343 State St,Rochester,NY,14650,2003-01-01T00:00:00.000,...,False,"{'type': 'Point', 'coordinates': [-77.61969, 4...",0000024-02-999-0000024-3100-3399-01-01-2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0000028-02-001,0000028,Ferrando Louise,3101 Laguna Apts,3101 Laguna St,San Francisco,CA,94123-3572,1993-09-30T00:00:00.000,...,False,"{'type': 'Point', 'coordinates': [-122.430984,...",0000028-02-001-0000028-5300-5399-09-30-1993,2,Marina,NaN,NaN,NaN,NaN,NaN


# Parsing locations into addresses 

Some of the movie filming locations aren't clear addresses, they'll require some cleaning. I intend to use fuzzy address matching to align the records

In [ ]:
all_unique_locations = list(df_movie_basic_info.loc[:, 'locations'].unique())
print('There are ' + str(len(all_unique_locations)) + ' unique locations listed in this dataframe. A sample of 10 values:')
pprint(all_unique_locations[0:10])

There are 1538 unique locations listed in this dataframe. A sample of 10 values:
['epic roasthouse (399 embarcadero)',
 'mason & california streets (nob hill)',
 'justin herman plaza',
 '200 block market street',
 'city hall',
 'polk & larkin streets',
 'randall museum',
 '555 market st.',
 'Empty',
 'the walden house, buena vista park']


In [ ]:
# collect all unique street names
all_unique_street_names = df_all_sf_address['street_name'].unique()  #for some reason this is NOT picking up the embarcadero 
print(f"Per OpenSF's database, there are {all_unique_street_names.shape[0]} unique street names in San Francisco.")
print(' 10 sample street names are: ', all_unique_street_names[0:10])

Per OpenSF's database, there are 1949 unique street names in San Francisco.
 10 sample street names are:  ['alemany' 'balboa' 'bayview park' 'berry' 'burke' 'donahue' 'earl'
 'edgehill' 'fitzgerald' 'gibson']


In [ ]:
unique_street_names_in_movie_location = dict()
to_unique_street_match = list()

for i, movie_location in tqdm.tqdm(enumerate(all_unique_locations)):
    matches =[street_name.lower() for street_name in all_unique_street_names if street_name.lower() in movie_location.lower()]
    unique_street_names_in_movie_location[movie_location] = matches

key_indexes = list(unique_street_names_in_movie_location)[3]

print('sample matching street name is', unique_street_names_in_movie_location[key_indexes])

1538it [00:00, 2184.37it/s]

sample matching street name is ['market', 'mark']


In [ ]:
# create boolean for rows of ALL SF STREETS that on the unique street name that is a substring of the movie location

df_relevant_sf_address = dict() #create dict where KEY = movie location, VALUE = a DF containing all street addresses on Street of interest (SOI)

for i, movie_location in tqdm.tqdm(enumerate(unique_street_names_in_movie_location)):
    df_relevant_sf_address[movie_location] = [] #in case you had to skip 
    
    if unique_street_names_in_movie_location[movie_location] == []:
        continue

    street_is_substring_bool =  df_all_sf_address.loc[:, 'street_name'].isin(unique_street_names_in_movie_location[movie_location]) #this is list of rows that are in 
    df_relevant_sf_address[movie_location] =df_all_sf_address.loc[street_is_substring_bool, :]

1538it [00:12, 122.14it/s]


In [ ]:
movie_original_location_dict = dict()
#iterate through ALL movie locations 
unique_street_names_in_movie_location
for iter_num,iter_location in tqdm.tqdm(enumerate(df_movie_basic_info.loc[:,'locations'])):
    movie_original_location_dict[iter_location] = 'Empty'

    if np.logical_or(iter_location == 'Empty', type(df_relevant_sf_address[iter_location]) is list): #it will be a list if no matches for that street address exist
        continue
    
    list_of_choices = list(df_relevant_sf_address[iter_location].loc[:,'address'].values) #index the dict of Dfs to find the df of RELEVANT addresses, turn that into list of values for comparison

    sample_match_highest = process.extractOne(iter_location, list_of_choices, scorer = fuzz.token_sort_ratio) #can pass scorer for extract One
    movie_original_location_dict[iter_location] = sample_match_highest[0] #create key value pair where key is OLD LOCATION, value is NEW location


1976it [00:23, 84.20it/s]


In [ ]:
df_movie_location_fuzz_guess = pd.DataFrame(movie_original_location_dict.items(), columns = ['Input Name', 'Best Guess'])
df_movie_location_fuzz_guess

,Input Name,Best Guess
0,epic roasthouse (399 embarcadero),399 the embarcadero
1,mason & california streets (nob hill),1 california st
2,justin herman plaza,1 justin dr
3,200 block market street,2200 b market st
4,city hall,Empty
...,...,...
1533,26 edna st,26 edna st
1534,119 vernon st,119 vernon st
1535,3202 folsom st,3202 folsom st
1536,110 pierce st,110 pierce st


In [ ]:
df_non_matched_locations = df_movie_location_fuzz_guess.loc[df_movie_location_fuzz_guess["Best Guess"] == "Empty", :]
df_non_matched_locations

,Input Name,Best Guess
4,city hall,Empty
8,Empty,Empty
15,coit tower,Empty
16,pier 50- end of the pier,Empty
33,palace of fine arts,Empty
...,...,...
1414,cow palace,Empty
1438,piers 30-32,Empty
1439,pier 24,Empty
1474,antonio street,Empty


#### Compare unmatched locations to major landmarks

In [ ]:
#list of choices will be the landmark wiki
df_non_matched_locations_guesses = dict()
#df_landmark_wiki.head()
list_of_choices = list(df_landmark_wiki.loc[:,'Landmark Name'].values) #index the dict of Dfs to find the df of RELEVANT addresses, turn that into list of values for comparison

for iter_num,iter_location in tqdm.tqdm(enumerate(df_non_matched_locations.loc[:,'Input Name'])):
    sample_match_highest = process.extractOne(iter_location, list_of_choices, scorer = fuzz.token_set_ratio) #can pass scorer for extract One
    df_non_matched_locations_guesses[iter_location]= sample_match_highest

df_non_matched_locations_guesses

81it [00:00, 187.27it/s]


{'"metzger\'s apt" 151 alice b. toklas pl.': ('alemany emergency hospital and health building',
  44),
 '145 9th st': ('i. m. scott school', 38),
 '145 barlett st.': ('haslett warehouse', 45),
 '2 rowland': ('koshland house', 52),
 '253 4th ave.': ('geary theater', 42),
 '2930 3rd street': ('ghirardelli annex-jackson street', 57),
 '386 4th ave': ('geary theater', 42),
 '44 montogomery': ('fairmont hotel', 43),
 '4704 3rd st': ('saint brigid church', 40),
 'Empty': ('stanyan house', 33),
 'administration building (treasure island)': ('kohl building', 76),
 'alamo square': ('ghirardelli square', 67),
 'alcatraz island': ('italian american bank', 50),
 'antonio alley': ('washington square', 53),
 'antonio street': ('ghirardelli annex-jackson street', 60),
 'areas around transamerica building': ('transamerica building (old fugazi bank building)',
  76),
 'at&t stadium': ("jack's restaurant", 48),
 'aub zam zam bar': ("sam jordan's bar", 59),
 'barbary coast': ('burr house', 52),
 'caffe t

because a lot of these guesses have a low similarity ratio (close to 50%), we threshold the ones below 90%

In [ ]:
#threshold highest matching tokens to extrapolate, cuting 
for key_val in list(df_non_matched_locations_guesses.keys()):
    if int(df_non_matched_locations_guesses[key_val][1]) < 90: #if exceeds threshold 
        df_non_matched_locations_guesses.pop(key_val, None) #delete entry
    else:
        df_non_matched_locations_guesses[key_val] = df_non_matched_locations_guesses[key_val][0]

In [ ]:
df_landmark_location_guess = pd.DataFrame(data = df_non_matched_locations_guesses.items(), columns = ['Input Name', 'Landmark Match'])
df_landmark_location_guess

,Input Name,Landmark Match
0,city hall,san francisco city hall
1,coit tower,lillie hitchcock coit tower
2,palace of fine arts,palace of fine arts
3,ferry building,ferry building
4,gabbiano's restaurant (ferry building),ferry building
5,the ferry building,ferry building


In [ ]:
for index, key in enumerate(df_non_matched_locations_guesses):
    print(index, key)
    df_non_matched_locations.loc[df_non_matched_locations['Input Name'] == key, 'Best Guess'] = df_non_matched_locations_guesses[key]

0 city hall
1 coit tower
2 palace of fine arts
3 ferry building
4 gabbiano's restaurant (ferry building)
5 the ferry building


In [ ]:
num_non_matched_locations = df_non_matched_locations.shape[0]
num_total_locations = len(unique_street_names_in_movie_location)
print( "Proportion of unmatched are ", round(num_non_matched_locations/num_total_locations, 2))

Proportion of unmatched are  0.05


Given a miss rate of about 5%, I'll continue with a preliminary mapping of the data. First though, I need to join the guesses with the original movie data.

In [ ]:
df_movie_location_fuzz_guess.head(1)


,Input Name,Best Guess
0,epic roasthouse (399 embarcadero),399 the embarcadero


In [ ]:

df_movie_basic_info.head(1)

,title,release_year,locations
0,180,2011,epic roasthouse (399 embarcadero)


In [ ]:
df_all_sf_address.head(1)

,index,eas_baseid,address,address_number,street_name,street_type,zip_code,cnn,longitude,latitude,point,supdist,supdistpad,numbertext,nhood,address_number_suffix
0,0,383793,0 alemany blvd,0,alemany,blvd,94110,2156000,-122.41038038,37.73526758,"{'type': 'Point', 'coordinates': [-122.4103803...",SUPERVISORIAL DISTRICT 10,10,TEN,Portola,NaN


In [ ]:
#join fuzzy guesses with the movie info
df_movies_best_guesses = df_movie_basic_info.merge(df_movie_location_fuzz_guess, how = 'left', left_on = 'locations', right_on = 'Input Name')

In [ ]:
#now join movie guesses with address info
df_movies_full_address = df_movies_best_guesses.merge(df_all_sf_address, how = 'left', left_on = 'Best Guess', right_on = 'address')
df_movies_full_address

,title,release_year,locations,Input Name,Best Guess,index,eas_baseid,address,address_number,street_name,...,zip_code,cnn,longitude,latitude,point,supdist,supdistpad,numbertext,nhood,address_number_suffix
0,180,2011,epic roasthouse (399 embarcadero),epic roasthouse (399 embarcadero),399 the embarcadero,3438.0,488092,399 the embarcadero,399,embarcadero,...,94105,12550202,-122.38903515,37.79046991,"{'type': 'Point', 'coordinates': [-122.3890351...",SUPERVISORIAL DISTRICT 6,06,SIX,Financial District/South Beach,NaN
1,180,2011,mason & california streets (nob hill),mason & california streets (nob hill),1 california st,3866.0,280800,1 california st,1,california,...,94111,3524000,-122.39704304,37.79321375,"{'type': 'Point', 'coordinates': [-122.3970430...",SUPERVISORIAL DISTRICT 6,06,SIX,Chinatown,NaN
2,180,2011,justin herman plaza,justin herman plaza,1 justin dr,4045.0,384248,1 justin dr,1,justin,...,94112,7718000,-122.42356684,37.73469234,"{'type': 'Point', 'coordinates': [-122.4235668...",SUPERVISORIAL DISTRICT 8,08,EIGHT,Outer Mission,NaN
3,180,2011,200 block market street,200 block market street,2200 b market st,317.0,486211,2200 b market st,2200,market,...,94114,8766201,-122.43135917,37.76584073,"{'type': 'Point', 'coordinates': [-122.4313591...",SUPERVISORIAL DISTRICT 5,05,FIVE,Haight Ashbury,B
4,180,2011,city hall,city hall,Empty,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,Women is Losers,2020,3202 folsom st,3202 folsom st,3202 folsom st,2913.0,484442,3202 folsom st,3202,folsom,...,94110,5703000,-122.41375188,37.74709098,"{'type': 'Point', 'coordinates': [-122.4137518...",SUPERVISORIAL DISTRICT 8,08,EIGHT,Mission,NaN
1973,Women is Losers,2020,110 pierce st,110 pierce st,110 pierce st,2487.0,289762,110 pierce st,110,pierce,...,94117,10418000,-122.43330082,37.77105605,"{'type': 'Point', 'coordinates': [-122.4333008...",SUPERVISORIAL DISTRICT 5,05,FIVE,Hayes Valley,NaN
1974,Women is Losers,2020,balmy alley,balmy alley,2 balmy st,993.0,275217,2 balmy st,2,balmy,...,94110,2699000,-122.41256654,37.75239947,"{'type': 'Point', 'coordinates': [-122.4125665...",SUPERVISORIAL DISTRICT 10,10,TEN,Bernal Heights,NaN
1975,"Yours, Mine and Ours",1968,Empty,Empty,Empty,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Mapping:

### Plotly Pass: 
I will use Plotly express,a high level API to access plotly functions

In [ ]:
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
#format for accessing an address lat and lon from the OG address book: df_all_sf_address.loc[1,'point']['coordinates']
df_movies_full_address.head(3)


,title,release_year,locations,Input Name,Best Guess,index,eas_baseid,address,address_number,street_name,...,zip_code,cnn,longitude,latitude,point,supdist,supdistpad,numbertext,nhood,address_number_suffix
0,180,2011,epic roasthouse (399 embarcadero),epic roasthouse (399 embarcadero),399 the embarcadero,3438.0,488092,399 the embarcadero,399,embarcadero,...,94105,12550202,-122.38903515,37.79046991,"{'type': 'Point', 'coordinates': [-122.3890351...",SUPERVISORIAL DISTRICT 6,06,SIX,Financial District/South Beach,NaN
1,180,2011,mason & california streets (nob hill),mason & california streets (nob hill),1 california st,3866.0,280800,1 california st,1,california,...,94111,3524000,-122.39704304,37.79321375,"{'type': 'Point', 'coordinates': [-122.3970430...",SUPERVISORIAL DISTRICT 6,06,SIX,Chinatown,NaN
2,180,2011,justin herman plaza,justin herman plaza,1 justin dr,4045.0,384248,1 justin dr,1,justin,...,94112,7718000,-122.42356684,37.73469234,"{'type': 'Point', 'coordinates': [-122.4235668...",SUPERVISORIAL DISTRICT 8,08,EIGHT,Outer Mission,NaN


In [ ]:
#import stored API key to access mapbox

from google.colab import drive
drive.mount('/content/drive')

with open('drive/MyDrive/Colab Notebooks/mapbox_token.txt', 'r') as f:
    mapbox_key = f.read().strip()
with open('drive/MyDrive/Colab Notebooks/omdb_api_key.txt', 'r') as omdb:
    omdb_key = omdb.read().strip()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#testing plotting scatterplot
df_movies_full_address.loc[:, ['longitude', 'latitude']] = df_movies_full_address.loc[:, ['longitude', 'latitude']].astype(float)
df_movies_full_address['release_year'] = df_movies_full_address['release_year'].astype(int)
df_movies_full_address['release_decade'] =np.floor(df_movies_full_address['release_year']/10)*10
df_movies_full_address['release_decade'] = df_movies_full_address['release_decade'].astype(int)
df_movies_full_address.head(2)

,title,release_year,locations,Input Name,Best Guess,index,eas_baseid,address,address_number,street_name,...,cnn,longitude,latitude,point,supdist,supdistpad,numbertext,nhood,address_number_suffix,release_decade
0,180,2011,epic roasthouse (399 embarcadero),epic roasthouse (399 embarcadero),399 the embarcadero,3438.0,488092,399 the embarcadero,399,embarcadero,...,12550202,-122.389035,37.790470,"{'type': 'Point', 'coordinates': [-122.3890351...",SUPERVISORIAL DISTRICT 6,06,SIX,Financial District/South Beach,NaN,2010
1,180,2011,mason & california streets (nob hill),mason & california streets (nob hill),1 california st,3866.0,280800,1 california st,1,california,...,3524000,-122.397043,37.793214,"{'type': 'Point', 'coordinates': [-122.3970430...",SUPERVISORIAL DISTRICT 6,06,SIX,Chinatown,NaN,2010


In [ ]:
sorted_year_list = np.sort(df_movies_full_address['release_year'].unique())
sorted_year_list
sorted_decade_list = np.sort(df_movies_full_address['release_decade'].unique())

In [ ]:
#https://plotly.com/python/plotly-express/
#https://medium.com/codex/bring-your-boring-plots-to-life-by-animating-them-acbb8b4b5388
#https://www.visualnoise.io/interactive-maps-with-python-pandas-and-plotly/
#https://plotly.com/python/scattermapbox/

#px.set_mapbox_access_token(mapbox_key)
fig = px.scatter_mapbox(df_movies_full_address, lon = "longitude", lat = "latitude",
                        zoom = 11, hover_name = 'title', hover_data = ['address', 'release_year'],
                        color = "release_decade", color_continuous_scale= 'jet'
                        , height = 600, width= 1000
                        )
fig.update_traces(marker=dict(size=5,  opacity = 0.33))
fig.update_layout(mapbox_style = 'open-street-map')
fig.show()

#animation_frame='release_year', category_orders = {'release_year':sorted_year_list}

### Import IMDB data to better sort datapoints
Going to be using the cinemagoer package (available at cinemagoer.github.io)

In [ ]:
#IMPORT IMDB datasets from https://cinemagoer.github.io/
movie_db = Cinemagoer()

#create df of movie information
all_movie_titles = df_movies_full_address['title'].unique().astype(list)
movie_years = df_movies_full_address.groupby('title')['release_year'].first().reset_index()

movie_years

,title,release_year
0,180,2011
1,24 Hours on Craigslist,2005
2,40 Days and 40 Nights,2002
3,48 Hours,1982
4,50 First Dates,2004
...,...,...
320,Woman on Top,2000
321,Woman on the Run,1950
322,Women is Losers,2020
323,"Yours, Mine and Ours",1968


In [ ]:
#use OMDB requests- 1000 requests per day https://www.omdbapi.com/
#OMDB api key is omdb_key

limit_request = 999

omdb_url =  f'http://www.omdbapi.com/?apikey={omdb_key}'
#set current query values and params
query_movie_year = movie_years.loc[111, 'release_year']
query_movie_title =movie_years.loc[111, 'title']
params = {"t": query_movie_title, "y":query_movie_year, "plot": 'full'}
#run current API request 
omdb_movie_search = requests.get(omdb_url, params = params).json()
omdb_movie_search.update({'release_year':query_movie_year, 'searched_title' : query_movie_title}) #add key value pairs that are the search terms used 
omdb_request_list.append(omdb_movie_search)

NameError: ignored

In [ ]:
#1.5 min for full loop ## FOR loop for OMDB request
omdb_request_list = list()
#use itertuples to iterate over rows of movie_years dataframe
for row in movie_years.itertuples(name = None):
    query_movie_year = row[2]
    query_movie_title =row[1]    
    params = {"t": query_movie_title, "y":query_movie_year, "plot": 'full'}
    #run current API request 
    omdb_movie_search = requests.get(omdb_url, params = params).json()
    omdb_movie_search.update({'release_year':query_movie_year, 'searched_title' : query_movie_title}) #add key value pairs that are the search terms used 
    omdb_request_list.append(omdb_movie_search)

In [ ]:
df_omdb_info = pd.DataFrame(omdb_request_list)
df_omdb_info #some rows are nan


In [ ]:
#download film info database
df_omdb_info.to_csv('drive/MyDrive/dataframe_omdb_info.csv')

In [ ]:
df_omdb_info['Genre'].unique()

#### MAP: Using open source maps to plot points on the map 
https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6

In [ ]:
##WIP: movie poster GAN
"""
https://www.omdbapi.com/ #has a poster API, 1000 api pull .
https://imdb-api.com/api # 100 api request/day
https://developers.themoviedb.org/3/getting-started/introduction #no rate limit stated 

#IDEA, make a GAN that creates FAKE MOVIE POSTERS FOR A REGION????? #is there enough input data 
https://www.johnkraszewski.com/media-synthesis
https://www.kaggle.com/neha1703/movie-genre-from-its-poster/version/3
https://github.com/benckx/dnn-movie-posters
https://github.com/siddtheshah/poster-gan"""